In [1]:
# converter of geochemical data for "endmembers" of specific to GHG rocks
# author:  thomas kosciuch 
# contact: thomas@kosciuch.ca
# affilite: University of Toronto - Earth Sciecnes


# 1. Directory Structure
#
#   python code
#           /XSLX/
#               xlsx files
# 
import scipy #dtypes
import numpy as np #dtypes
import pandas as pd
import os as os            #checks if directories exist for file writing
import mysql.connector

from numpy import linalg
import sys

#converts to CSV
import xlrd
import csv

## Custom packages 
sys.path.append(os.path.join(sys.path[0],'mods'))

import metamathicCalc as calc
import loader as loader
import matrixOps as Ops

#loads inputs / database 
notKnown, minerals, thermody = loader.load() 

In [2]:
# Index ## mineral selection
# minerals.loc[[54]]

# first we do manual entry, then we iterate:
print("how many minerals?")
minNum = int(input())
mins = [0] * minNum
for i in range(minNum):
    print("what is the No. of mineral ",+(i+1),"?")    
    mins[i] = int(input())

#converting to number sys in Excel
mins = [x-1 for x in mins]


how many minerals?
0


In [3]:
mins = [57, 138, 143, 105]
minNum = len(mins)
d = loader.minerals(mins, minerals)

In [4]:
mins

[57, 138, 143, 105]

In [5]:
# Initial Conditions
T_min   = 200
T_max   = 1000
P       = 2                         # [kbar]
res     = 101                       # resolution
TRange  = [T_min, T_max]
XCO2    = np.linspace(0,0.5, res)   # sets up plotting 
XCO2    = XCO2[1:res]               # removes final CO2 point
Phase   = mins                      # Sets up "Phase" variable to mimic MatLab
T       = [0] * res                 # creates empty matrix
# First guess for temperature
T_guess = (TRange[0]+TRange[1])/2   # guesses halfway between range
Xs      = [] 
CC=['  SiO2   ','  TiO2   ',' Al2O3   ','   FeO   ','   MgO   ','   MnO   ','   CaO   ','  Na2O   ','   K2O   ','     O   ','   H2O   ','   CO2   ',' Supl1   ',' Supl2   ']


In [6]:
# CALC UNI T AT P  - ITERATIVE
# Counts the number of 'end-members' each phase has

PHASE         = d
endnum        = np.zeros(minNum+2)
for i in range(0, minNum+2):
    target    = 'Mineral_' +  str(i)
    temp      = d.get(target)
    endnum[i] = len(temp.get('endmember'))

In [8]:
A = Ops.compress(d, PHASE, CC)


array([[8., 5., 2., 1., 0.],
       [0., 0., 1., 0., 1.],
       [0., 1., 1., 0., 2.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [22]:
def ToReducedRowEchelonForm( M):
    if not M: return
    lead = 0
    rowCount = len(M)
    columnCount = len(M[0])
    for r in range(rowCount):
        if lead >= columnCount:
            return
        i = r
        while M[i][lead] == 0:
            i += 1
            if i == rowCount:
                i = r
                lead += 1
                if columnCount == lead:
                    return
        M[i],M[r] = M[r],M[i]
        lv = M[r][lead]
        M[r] = [ mrx / float(lv) for mrx in M[r]]
        for i in range(rowCount):
            if i != r:
                lv = M[i][lead]
                M[i] = [ iv - lv*rv for rv,iv in zip(M[r],M[i])]
        lead += 1
 
 
mtx = [
   [ 1, 2, -1, -4],
   [ 2, 3, -1, -11],
   [-2, 0, -3, 22],]
 
print(A, mtx)
ToReducedRowEchelonForm( A )



[[8. 5. 2. 1. 0.]
 [0. 0. 1. 0. 1.]
 [0. 1. 1. 0. 2.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]] [[1, 2, -1, -4], [2, 3, -1, -11], [-2, 0, -3, 22]]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [27]:
# NOW function [React,A0,C]=CalcMatrix(phase)
"""% construct linear metamorphic reactions with the input phases.
% =======================================
% phase is the data-structure for solid solution models.
% A0 is the composition matrix before Gaussian elimination
% C is the Modal composition"""

# setup empty matrix (reserve memory):
A                       = np.zeros(((len(PHASE)), len(CC)))
minPres                 = []                    # min names

# Compositional matrix
# Col 1: 15 = CC
for i in range((len(PHASE))):
    for j in range(len(CC)-2):                  # -2 b/c of H2O+CO2
        target          = 'Mineral_' +  str(i)
        temp            = d.get(target)
        temp            = temp.get('SiO2toCO2')
        A[i][j]         = int(temp[j])
        temp            = d.get(target)
        temp            = temp.get('min_name')
    minPres.append(temp)                        # tracking mins

# Deletes zero columns
components = []
for i in range(len(CC)-1):
    target = len(CC) - 1 - i
    if sum(A[:,target]) > 0:               # Preserves present components
        components.insert(0,(CC[target]))    
    if sum(A[:,target]) == 0:
        A = np.delete(A,target,1)

A = [*zip(*A)]                             # Transpose. I think this helps for solving
A0 = A                                     # Copies A

for i in range(len(components)):
    print("compenents present: " + components[i])
    

compenents present:    MgO   
compenents present:    CaO   
compenents present:    H2O   
compenents present:    CO2   


In [12]:
b = [0, 0, 0, 0, 0]
ASOLVE = A
#ASOLVE = np.delete(ASOLVE,2,1)
#print(ASOLVE)
#np.linalg.solve(ASOLVE,b)

In [28]:
 minPres

['carbonDioxide', 'tremolite', 'calcite', 'dolomite', 'quartz', 'Water']

In [ ]:
if(dtype(agg[y]) == 'string'):

In [28]:
import sys

con = None

try:
    con = sqlite3.connect('test.db')

    cur = con.cursor()
    cur.execute('SELECT SQLITE_VERSION()')

    data = cur.fetchone()

    print "SQLite version: {}".format(data)

except sqlite3.Error, e:

    print "Error {}:".format(e.args[0])
    sys.exit(1)

finally:

    if con:
        con.close()  
                     

SyntaxError: invalid syntax (<ipython-input-28-9cb66b75d9f3>, line 13)

In [6]:
# 1. create sql struct per file

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        conn.close()


In [ ]:
conn = sqlite3.connect('mineralEndmembers.db')
c = conn.cursor()
# Create table
c.execute('''CREATE TABLE stocks
             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()
conn.close()

In [ ]:
variable = ["a", "b"]
len(variable)

In [ ]:
# f= open("guru99.txt","w+")

[56, 137, 142, 104]

In [27]:
 
mtx = [ 
   [ 1, 2, -1, -4],
   [ 2, 3, -1, -11],
   [-2, 0, -3, 22],]

In [25]:
calc.rref(mtx)
#for rw in mtx:
#    print ', '.join( (str(rv) for rv in rw) )

In [22]:
print(calc.rref(mtx))  # reduced row echalon

None


In [28]:
mtx

[[1, 2, -1, -4], [2, 3, -1, -11], [-2, 0, -3, 22]]